In [ ]:
pip install cdsapi

In [ ]:
import pandas as pd
import numpy as np
from geopy.distance import great_circle as vc 
import cdsapi
from datetime import datetime
#import xlsxwriter 
import csv
import math
import os
import datetime
BASIN = 'EP' # set WP, SP, EP, NI, SI
path = os.getcwd() 
path1 = path + '/'+BASIN+'/'

In [ ]:
data = pd.read_csv(path + '/ibtracs.since1980.list.v04r00.csv')
data1 = data[data['BASIN'] == BASIN]
data2 = data1[['SID', 'ISO_TIME','LAT','LON','WMO_WIND','WMO_PRES','DIST2LAND']]
if not os.path.exists(path1):
    os.makedirs(path1)

In [ ]:
d2lIndex = data2.columns.get_loc('DIST2LAND')
sidIndex = data2.columns.get_loc('SID')
sids = pd.unique(data2['SID'])
alldata = []
for sid in sids:
  cyclonData = data2[data2['SID'] == sid]
  valuesC = cyclonData.values
  length = len(valuesC)
  j = length -1
  sidChange = False # Giving a differnce cyclone ID in case cyclone crosses from ocean to land  multiple times
  # Starting from the last entry for a cyclone 
  while j >= 0:
    d2l = int(valuesC[j][d2lIndex])
    #Locating the movement of cyclone from ocean to land
    while d2l > 0 and j >=0:
      j = j-1
      d2l = int(valuesC[j][d2lIndex])
        
    #The above loop, for first time of its run will ignore those entries of cyclone that does not cross over coast after being over ocean
        
    #Ignoring the appearnace of cyclone on land
    while d2l == 0 and j>=0:
      j = j-1
      d2l = int(valuesC[j][d2lIndex])

    i = j # j is the first index when a cylcone moves from ocean to land. 
    
    #Now capturing all those entries of cyclone that are over ocean. 
    while d2l > 0 and i >=0:
      i = i-1
      d2l = int(valuesC[i][d2lIndex])

    if sidChange:
      newSID = sid+'_'+str(i+1) 

    #Now storing all the entires of cyclone over ocean to first time it touch coastal region. 
    #Also we are capturing only those entries over ocean that are 12 hour before the landfall. 
    for k in range(i+1,j-2,1):
      val = valuesC[k]
      if sidChange:
        val[sidIndex] = newSID
      alldata.append(val)

    val = valuesC[j+1]
    if sidChange:
      val[sidIndex] = newSID
    alldata.append(val)

    sidChange = True # The next loop will detect further movement of cyclone from ocean to land. Next such movement is given different SID to trat it as different cyclone. 
    j = i

dataa = pd.DataFrame(alldata, columns = ['SID', 'ISO_TIME','LAT','LON','WMO_WIND','WMO_PRES','DIST2LAND'])
dataa.to_csv(path1+BASIN+'OceanTillLandfall.csv')  

In [ ]:
import os
import os.path
#Function to check if a number if float or not
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

spread = 4 # Degree spread around center of cyclone

#Function to get u, v, wind and z geopotenital at three pressure levels  for a grid centerd at cyclone location. 
def getPressureData(cdsApi, year, month, date, time, lati, long, path):
  time1 = time.replace(':','_')
  fileName = path+ date +month+year+'_'+time1+'.grib'
  print('fileName  ', fileName)
  if os.path.isfile(fileName):
    print("file already exist")
    return
  cdsApi.retrieve(
    'reanalysis-era5-pressure-levels',
    {
        'product_type': 'reanalysis',
        'variable': [
             'geopotential', 'u_component_of_wind', 'v_component_of_wind',
        ],
        'pressure_level': [
            '225', '500', '700',
        ],
        'year': [
            year,
        ],
        'month': [
            month,
        ],
        'day': [
            date,
        ],
        'time': [
            time,
        ],
        'format': 'grib',
        'area': [
            lati+spread, long-spread, lati-spread,
            long+spread,
        ],
    },
    fileName)
  
#Function to get SST data for grid centerd at cyclone center
def getSSTData(cdsApi, year, month, date, time, lati, long, path):
  time1 = time.replace(':','_')
  fileName = path+ date +month+year+'_'+time1+'.grib'
  print('fileName  ', fileName)
  if os.path.isfile(fileName):
    print("file already exist")
    return
  cdsApi.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'grib',
        'variable': 'sea_surface_temperature',
        'year': year,
        'month': month,
        'day': date,
        'time': time,
        'area': [
            lati+spread, long-spread, lati-spread,
            long+spread,
        ],
    },
    fileName)
  
def createFolder(path):
  if not os.path.exists(path):
    os.makedirs(path)
    
#Rounding lat, long to the multiple of 0.25
def roundF(x, base=.25):
    return base * round(x/base)

In [ ]:
data2 = pd.read_csv(path1+BASIN+'OceanTillLandfall.csv', index_col=0)
sidIndex = data2.columns.get_loc('SID') 
isoTimeIndex = data2.columns.get_loc('ISO_TIME') 
latIndex = data2.columns.get_loc('LAT') 
longIndex = data2.columns.get_loc('LON')
count = 0
dataValues = data2.values
cdsApi = cdsapi.Client()
for data in dataValues:
  print('count  ', count)
  count = count + 1
  sid = data[sidIndex]
  #Creating folder for pressure data
  folder1 = path1+'data/pressure/'+sid+'/'
  #Creating folder for SST data
  folder2 = path1 +'data/sst/'+sid+'/'
  createFolder(folder1)
  createFolder(folder2)
  isoTime = data[isoTimeIndex]
  lat = data[latIndex]
  long = data[longIndex]
  date = datetime.datetime.strptime(isoTime, '%Y-%m-%d %H:%M:%S')
  year = date.year

  year = str(year)
  month = date.month
  day = date.day
  time = date.hour

  if month < 10:
    month = '0'+str(month)
  else:
    month = str(month)

  if day < 10:
    day = '0'+str(day)
  else:
    day = str(day)

  if time<10:
    time = '0'+str(time)+':00'
  else:
    time = str(time) +':00'

  lat = roundF(lat)
  long = roundF(long)

  #print('day  ', day, ' month ', month, ' year ', year, ' time ', time, ' lat ', lat, ' long ', long)
  getPressureData(cdsApi, year, month, day, time, lat, long, folder1)
  getSSTData(cdsApi, year, month, day, time, lat, long, folder2)

In [ ]:
#Function that return the .grib file.
def getFileName(folder, isoTime):
    date = datetime.datetime.strptime(isoTime, '%Y-%m-%d %H:%M:%S')

    year = date.year
    year = str(year)
    month = date.month
    day = date.day
    time = date.hour

    if month < 10:
      month = '0'+str(month)
    else:
      month = str(month)

    if day < 10:
      day = '0'+str(day)
    else:
      day = str(day)

    if time<10:
      time = '0'+str(time)+'_00'
    else:
      time = str(time) +'_00'

    file1 = folder+ day +month+year+'_'+time+'.grib'
    return file1

In [ ]:
pip install eccodes

In [ ]:
#Reading .grib files and extracting u, v, z fileds and SST for 33*33 grid.
#For each filed we have created a column for field values at each cell of grid. So for each fields we have 33*33 = 1089 columns.
import eccodes
from eccodes import *
data2 = pd.read_csv(path1+BASIN+'OceanTillLandfall.csv', index_col = 0, header=0)

sidIndex = data2.columns.get_loc('SID') 
isoTimeIndex = data2.columns.get_loc('ISO_TIME') 
latIndex = data2.columns.get_loc('LAT') 
longIndex = data2.columns.get_loc('LON') 
wwindIndex = data2.columns.get_loc('WMO_WIND') 
wpresIndex = data2.columns.get_loc('WMO_PRES') 
d2lIndex = data2.columns.get_loc('DIST2LAND')

sids = pd.unique(data2['SID'])
gridSize = 33
alldata = []

colums = ['SID', 'ISO_TIME','LAT','LON','WMO_WIND','WMO_PRES','DIST2LAND']
check = True
for sid in sids:
  cyclonData = data2[data2['SID'] == sid]
  valuesC = cyclonData.values
  length = len(valuesC)

  folder1 = path1 +'/data/pressure/'+sid+'/'
  folder2 = path1 +'/data/sst/'+sid+'/'
  for i in range(length):
    #print("i   ", i)
    datac = []
    data = valuesC[i]
    isoTime = data[isoTimeIndex]
    lat = data[latIndex]
    long = data[longIndex]
    wwind = data[wwindIndex]
    wpres = data[wpresIndex]
    d2l = data[d2lIndex]

    datac.append(sid)
    datac.append(isoTime)
    datac.append(lat)
    datac.append(long)
    datac.append(wwind)
    datac.append(wpres)
    datac.append(d2l)

    pressureFile = getFileName(folder1, isoTime)
    sstFile = getFileName(folder2, isoTime)
    
    #print('pressureFile  ',pressureFile)
    
    id = codes_index_new_from_file(pressureFile, ["shortName",'level'])
    id1 = codes_index_new_from_file(sstFile, ["shortName"])

    names = codes_index_get(id, "shortName")
    levels = codes_index_get(id, "level")

    for name in names:
      #Getting index for each name (u, v, z)
      codes_index_select(id, "shortName", name)
      for level in levels:
        #Getting index for each leval 225, 500, 700
        codes_index_select(id, "level", level)
        gid = codes_new_from_index(id)
        values = codes_get_values(gid)
        length = len(values)
        # Appening values of field for each location of grid. 
        for i in range(length):
            val = np.float32(values[i])
            datac.append(val)
            #Creating dataset columnds name
            if check:
                colums.append(name+str(level)+'_'+str(i))

    #Getting index for SST. 
    codes_index_select(id1, "shortName", 'sst')
    gid1 = codes_new_from_index(id)
    values = codes_get_values(gid)
    length = len(values)
    #Extracting SST for each locatoin of grid. 
    for i in range(length):
        val = np.float32(values[i])
        datac.append(val)
        if check:
            colums.append('sst'+'_'+str(i))
    alldata.append(datac)
    codes_index_release(id)
    codes_index_release(id1)
    check = False
  
dataa = pd.DataFrame(alldata, columns = colums)
dataa.to_csv(path1+BASIN+'pressureSSTData.csv')